In [1]:
# Importação das bibliotecas necessárias
import numpy as np

# 1. Lê o arquivo e pega as dimensões 

In [2]:
# Transformando os dados
def juntaParMaquinaTempoExecução(vetor):
    resultado = []
    tamanho = len(vetor)
    i = 0
    while i < tamanho:
        resultado.append([vetor[i],vetor[i+1]])
        i = i + 2
    return resultado

def converteArrayEmMatriz(data):
    # Converte o array em uma matriz
    dadosFormatados = []
    for item in data:
        dadosFormatados.append(juntaParMaquinaTempoExecução(item.split()))
    return dadosFormatados

In [51]:
# Função de aptidão
def funcaoAptidao(posicaoCandidata, matrizOperacoesTempo):
    """
        Função para avaliar o desempenho do algoritmo
            Parâmetros:
                - posicaoCandidata: Vetor de N tarefas x M máquinas. Exemplo: [O11, 021,..]
                - matrizOperacoesTempo: Matriz N tarefas x M máquinas, onde cada posição é um array contendo a Máquina e seu Tempo. Exemplo: [['1', '20'], ['0', '15'],..]
    """
    numeroTarefas = len(matrizOperacoesTempo)
    numeroMaquinas = len(matrizOperacoesTempo[0])
    vetorTempoMaquinasEmExecucao = np.zeros(numeroMaquinas)
    vetorTarefasEmExecucao = np.zeros(numeroTarefas)
    # i[0] -> TAREFA
    # i[1] -> OPERACAO
    for i in posicaoCandidata:
        # print(i[0])
        # print(i[1])
        # print(matrizOperacoesTempo[int(i[0]) - 1 ][int(i[1]) - 1])
        maquina = int(matrizOperacoesTempo[int(i[0]) - 1 ][int(i[1]) - 1][0])
        tempoMaquina = int(matrizOperacoesTempo[int(i[0]) - 1][int(i[1]) - 1][1])
        tempoAtualAuxiliar = max(vetorTempoMaquinasEmExecucao[maquina - 1], vetorTarefasEmExecucao[int(i[0]) - 1])
        vetorTempoMaquinasEmExecucao[maquina - 1] = tempoAtualAuxiliar + tempoMaquina
        vetorTarefasEmExecucao[int(i[0]) - 1] = max(vetorTempoMaquinasEmExecucao[maquina - 1], vetorTarefasEmExecucao[int(i[0]) - 1])
    return [max(vetorTempoMaquinasEmExecucao), np.argmax(vetorTempoMaquinasEmExecucao) + 1]

In [ ]:
resultado = [50, 18, 3, 20, 49, 15, 42, 7, 24, 30, 9, 27, 1, 44, 34, 8, 2, 41, 37, 16, 21, 32, 46, 25, 38, 26, 43, 40, 12, 28, 35, 36, 47, 11, 22, 17, 13, 48, 6, 23, 10, 5, 45, 39, 19, 29, 14, 4, 31, 33]
[['6', '1'], ['2', '1'], ['1', '1'], ['3', '1'], ['5', '1'], ['2', '2'], ['5', '2'], ['1', '2'], ['3', '2'], ['4', '1'], ['1', '3'], ['3', '3'], ['1', '4'], ['5', '3'], ['4', '2'], ['1', '5'], ['1', '6']]

In [139]:
np.floor(49/5)

9.0

In [52]:
# Função de decoding
def decodingValoresReaisParaOperacoes(vetorValoresReais, numeroTarefas, numeroMaquinas):
    """
        Função para decodificar um array de valores reais em operações
            Parâmetros:
                - vetorValoresReais: Vetor com valores reais de dimensão N tarefas x M máquinas. Exemplo: [0.98, 0.11,..]
                - numeroTarefas: Número de tarefas do problema. 
                - numeroMaquinas: Número de máquinas do problema. 
    """
    # print(vetorValoresReais)
    vetorAuxiliarOrdenado = vetorValoresReais.copy()
    vetorAuxiliarOrdenado.sort()
    resultado = []
    matrizOperacoes = []
    for item in vetorValoresReais:
        resultado.append(list(vetorAuxiliarOrdenado).index(item) + 1)
    # print('resultado: {}'.format(resultado))
    countAux = np.ones(numeroTarefas)
    for item in resultado:
        contaArredondada = round((np.floor((item-1)/ numeroMaquinas)))
        matrizOperacoes.append([str(contaArredondada + 1), str(round(countAux[contaArredondada]))])
        countAux[contaArredondada] += 1
    return matrizOperacoes

In [5]:
# Função VND - Shift
def VND_shift(posicaoCandidata):
    """
        VND com estrutura de vizinhança Shift (Realoca a posição de uma operação numa mesma máquina)
            Parâmetros:
                - posicaoCandidata: Vetor com valores reais de dimensão N tarefas x M máquinas. Exemplo: [0.98, 0.11,..]
    """
    # print(posicaoCandidata)
    copiaPosicaoCandidata = posicaoCandidata.copy()
    [posicaoElementoASerDeslocado, posicaoFutura] = np.random.randint(1, len(copiaPosicaoCandidata), 2)
    # print(posicaoElementoASerDeslocado)
    # print(posicaoFutura)
    elementoASerDeslocado = copiaPosicaoCandidata[posicaoElementoASerDeslocado]
    del(copiaPosicaoCandidata[posicaoElementoASerDeslocado])
    primeiraParteVetor = copiaPosicaoCandidata[:posicaoFutura]
    segundaParteVetor = copiaPosicaoCandidata[posicaoFutura:]
    return primeiraParteVetor + [elementoASerDeslocado] + segundaParteVetor
    
# Função VND - Switch
def VND_switch(posicaoCandidata):
    """
        VND com estrutura de vizinhança Switch (Troca a posição de duas operações numa mesma máquina)
            Parâmetros:
                - posicaoCandidata: Vetor com valores reais de dimensão N tarefas x M máquinas. Exemplo: [0.98, 0.11,..]
    """
    # print(posicaoCandidata)
    copiaPosicaoCandidata = posicaoCandidata.copy()
    [primeiraPosicao, segundaPosicao] = np.random.randint(1, len(copiaPosicaoCandidata), 2)
    # print(primeiraPosicao)
    # print(segundaPosicao)
    primeiroElemento = copiaPosicaoCandidata[primeiraPosicao]
    copiaPosicaoCandidata[primeiraPosicao] = copiaPosicaoCandidata[segundaPosicao]
    copiaPosicaoCandidata[segundaPosicao] = primeiroElemento
    return copiaPosicaoCandidata

# print('VND Switch')
# print(VND_switch(posicaoCandidataComValoresReais))

# print('')
# print('VND Shift')
# print(VND_shift(posicaoCandidataComValoresReais))

In [6]:
def particleSwarmOptimizationMethod(positionVector, velocityVector, bestPersonal, bestGlobal, c1, c2, w):
    """
        Particle Swarm Optimization Method
            Parameters:
            - positionVector: Array that contains real values with D positions
            - velocityVector: Array that contains real values with D positions
            - bestPersonal: Real values array from the best personal position evaluated
            - globalPersonal: Real values array from the global personal position evaluated
            - c1: Constant
            - c2: Constant
            - w: Constant
    """
    personalFunction = c1 * np.random.rand() * (bestPersonal - positionVector)
    globalFunction = c2 * np.random.rand() * (bestGlobal - positionVector)
    nextVelocityVector = w * velocityVector + personalFunction + globalFunction
    nextPositionVector = positionVector + nextVelocityVector
    return [nextPositionVector, nextVelocityVector]

In [7]:
with open('./instances/la01.txt') as file:
    lines = file.read()

dimensions = lines.split('\n')[0]
data = lines.split('\n')[1:]

tasks = int(dimensions.split()[0])
machines = int(dimensions.split()[1])
print('Número de tasks: ', tasks)
print('Número de máquinas: ', machines)

Número de tasks:  10
Número de máquinas:  5


In [8]:
# Format data and print
dadosFormatados = converteArrayEmMatriz(data)
print(dadosFormatados)

# Printa para ver o resultado
for i in range(0, len(dadosFormatados)):
    print("Tarefa {}: {}".format(i+1, dadosFormatados[i]))

print("")
print("Exemplo para facilitar o entendimento")
print("Tarefa {}: {}".format(1, dadosFormatados[0]))
for i in range(0, len(dadosFormatados[0])):
    print("Tarefa {} -> Operação 1,{}: {} -> Máquina {} com tempo {}".format(i+1, i+1, dadosFormatados[0][i], dadosFormatados[0][i][0], dadosFormatados[0][i][1]))

[[['1', '21'], ['0', '53'], ['4', '95'], ['3', '55'], ['2', '34']], [['0', '21'], ['3', '52'], ['4', '16'], ['2', '26'], ['1', '71']], [['3', '39'], ['4', '98'], ['1', '42'], ['2', '31'], ['0', '12']], [['1', '77'], ['0', '55'], ['4', '79'], ['2', '66'], ['3', '77']], [['0', '83'], ['3', '34'], ['2', '64'], ['1', '19'], ['4', '37']], [['1', '54'], ['2', '43'], ['4', '79'], ['0', '92'], ['3', '62']], [['3', '69'], ['4', '77'], ['1', '87'], ['2', '87'], ['0', '93']], [['2', '38'], ['0', '60'], ['1', '41'], ['3', '24'], ['4', '83']], [['3', '17'], ['1', '49'], ['4', '25'], ['0', '44'], ['2', '98']], [['4', '77'], ['3', '79'], ['2', '43'], ['1', '75'], ['0', '96']]]
Tarefa 1: [['1', '21'], ['0', '53'], ['4', '95'], ['3', '55'], ['2', '34']]
Tarefa 2: [['0', '21'], ['3', '52'], ['4', '16'], ['2', '26'], ['1', '71']]
Tarefa 3: [['3', '39'], ['4', '98'], ['1', '42'], ['2', '31'], ['0', '12']]
Tarefa 4: [['1', '77'], ['0', '55'], ['4', '79'], ['2', '66'], ['3', '77']]
Tarefa 5: [['0', '83'], [

In [115]:
# Dados mockados para teste
posicaoCandidataTeste = [['1','1'], ['3', '1'], ['2', '1'], ['3', '2'], ['1', '2'], ['2', '2'], ['3', '3'], ['2', '3'], ['1', '3']]
matrizOperacoesTempoTeste = [[['1', '2'], ['2', '3'], ['3', '1']], [['2', '3'], ['1', '1'], ['3', '2']], [['2', '2'], ['3', '3'], ['1', '2']]]

# Dados mockados para decoding
posicaoCandidataComValoresReais = [0.13, 0.75, 0.48, 0.57, 0.21, 0.37, 0.96, 0.42, 0.06]

In [32]:
# Mock de dados para Teste PSO
numberOfParticles = 10
dimension = tasks*machines
c1 = 1
c2 = 1
w = 1

candidatePositionMatrix = np.zeros((numberOfParticles, dimension))
candidateVelocityMatrix = np.zeros((numberOfParticles, dimension))
for i in range(0, numberOfParticles):
    candidatePositionMatrix[i] = np.random.rand(dimension)
    candidateVelocityMatrix[i] = np.random.randn(dimension)

# print('Vetor Posicao Inicial {} x {}'.format(N, D))
# print(candidatePositionMatrix)
# print('')
# print('Vetor Velocidade Inicial {} x {}'.format(N, D))
# print(candidateVelocityMatrix)

In [56]:
# Number of iterations
iterations = 10

# Set  
positionMatrixAux = candidatePositionMatrix
velocityMatrixAux = candidateVelocityMatrix

# Initialize vectors
bestPersonalPositionVector = np.zeros((numberOfParticles, dimension))
bestPersonalEvaluatedValue = np.zeros((numberOfParticles, dimension))

# Set initial values of vectors
for i in range(0, numberOfParticles):
    bestPersonalPositionVector[i] = candidatePositionMatrix[i]
    # print(bestPersonalPositionVector[i])
    decodedArray = decodingValoresReaisParaOperacoes(bestPersonalPositionVector[i], tasks, machines)
    # print(decodedArray)
    bestPersonalEvaluatedValue[i] = funcaoAptidao(decodedArray, dadosFormatados)[0]

# Find the best fitness value (the smallest value)
indexOfBetterFitness = np.argmin(bestPersonalEvaluatedValue)
valueBetterFitness = np.min(bestPersonalEvaluatedValue)
# print(bestPersonalEvaluatedValue)
# print(valueBetterFitness)
bestGlobalPositionVector = candidatePositionMatrix[round(indexOfBetterFitness/50)]


In [57]:

# Method Proposed
for it in range(0, iterations):
    for amostra in range(0, numberOfParticles):
        # Run PSO
        [newPositionVector, newVelocityVector] = particleSwarmOptimizationMethod(positionMatrixAux[amostra], velocityMatrixAux[amostra], bestPersonalPositionVector[amostra], bestGlobalPositionVector, c1, c2, w)
        # Apply decoding funtion to convert real array numbers to operations array
        operationsConverted = decodingValoresReaisParaOperacoes(newPositionVector, dimension)
        # Evaluate the fitness from operations array found
        [fitness, finalMachine] = funcaoAptidao(operationsConverted, dadosFormatados)
        # Compare values to update the best fitness
        if(fitness < bestPersonalEvaluatedValue[amostra]):
            bestPersonalEvaluatedValue[amostra] = fitness
        positionMatrixAux[amostra] = newPositionVector
        velocityMatrixAux[amostra] = newVelocityVector
    indexOfBetterFitness = np.argmin(bestPersonalEvaluatedValue)
    bestGlobalPositionVector = candidatePositionMatrix[indexOfBetterFitness]
# bestGlobalPositionVector

TypeError: decodingValoresReaisParaOperacoes() missing 1 required positional argument: 'numeroMaquinas'